In [2]:
using Random
using DataFrames
using CSV

In [4]:
function build_lattice_df(Lx,Ly)
    
    N = Lx*Ly
    interactions = DataFrame(["$spin" => [] for spin in 1:N])    
    for spin in 1:N
        up = mod(spin-1-Lx,N)
        down = mod(spin-1+Lx,N)
        right = Int(((spin-1)-((spin-1)%Lx)) + mod(((spin-1)%Lx)+1,Lx))
        left = Int(((spin-1)-((spin-1)%Lx)) + mod(((spin-1)%Lx)-1,Lx))
        push!(interactions[!,"$spin"],[up+1,down+1,left+1,right+1])
    end
    
    return interactions
end

build_lattice_df (generic function with 1 method)

In [13]:
function Calculate_Ising_Energy(config,interactions,J)
    
    N = length(config)
    energy = 0
    
    for spin in (1:N)
        neighbors = interactions[!,"$spin"][1,1]
        energy_contributions = 0.5*-1*J*config[spin].*config[neighbors]
        energy += sum(energy_contributions)
    end
    
    return energy
    
end

Calculate_Ising_Energy (generic function with 1 method)

In [4]:
function build_deltaE_df(beta)
    
    possible_sum_neighbors = collect(-4:2:4)
    spins = [1,-1]
    deltaEs = DataFrame(["$spin,$poss" => [] for poss in possible_sum_neighbors for spin in spins])    
    for sum_neighbors in possible_sum_neighbors
        spinup_deltaE = 2*sum_neighbors
        spindown_deltaE = -2*sum_neighbors 
        push!(deltaEs[!,"1,$sum_neighbors"],spinup_deltaE)
        push!(deltaEs[!,"-1,$sum_neighbors"],spindown_deltaE)
    end
    
    return deltaEs
end

build_deltaE_df (generic function with 1 method)

In [5]:
function Metropolis_update_function(config,interactions,beta,J,deltaE_vals)
    
    rand_spin_flip_index = rand(1:length(config),1)[1]
    new_config = copy(config)

    spin = Int(config[rand_spin_flip_index])
    neighbors = interactions[!,"$rand_spin_flip_index"][1,1]
    sum_neighbors = Int(sum(config[neighbors]))
    deltaE = deltaE_vals[!,"$spin,$sum_neighbors"][1]
    exp_beta_deltaE = exp(-1*beta*deltaE)
    rand_value = rand(Float64,1)[1]
    
    if exp_beta_deltaE > rand_value
        new_config[rand_spin_flip_index] = -1*config[rand_spin_flip_index]
    else
        deltaE = 0.
    end
    
    return new_config, deltaE
end

Metropolis_update_function (generic function with 1 method)

In [51]:
function MonteCarlo_Ising(beta,J,Lx,Ly,warmup_steps,steps,steps_per_step,build_lattice_function,build_update_probs,update_function,energy_function)
    
    # system information
    N = Lx*Ly
    interactions = build_lattice_function(Lx,Ly)
    deltaEs = build_deltaE_df(beta)
    initial_config = rand((-1,1),N)
    
    # initialize dataframes
#     fulldf = DataFrame(:Es => Float64[],:E2s => Float64[],:Ms => Int[],:M2s => Int[])
    maindf = DataFrame(:avenergy => Float64[],:avenergy2 => Float64[],:avM => Float64[],:avM2 => Float64[])
    
    # warmup steps - track Es and Ms but not in averages
    config = initial_config
    energy = energy_function(config,interactions,J)
    for step in (1:warmup_steps)
        for inner_steps in (1:steps_per_step)
            new_config, deltaE = update_function(config,interactions,beta,J,deltaEs)
            config = copy(new_config)
            new_energy = energy+deltaE
            energy = new_energy
#             M = sum(new_config)
        end
#         push!(fulldf,(energy,energy^2,M,M^2))
    end
    
    # begin MC steps and observable tracking
    sumE = 0
    sumE2 = 0
    sumM = 0
    sumM2 = 0
    println("Warmup steps over!")
#     println(energy)
    for step in (1:steps)
        for inner_step in (1:steps_per_step)
            new_config,deltaE = update_function(config,interactions,beta,J,deltaEs)
            config = copy(new_config)
            new_energy = energy+deltaE
            energy = new_energy
        end
        M = sum(config)
#         push!(fulldf,(new_energy,new_energy^2,M,M^2))
        absM = abs(M)
        sumE = sumE+energy
        sumE2 = sumE2+energy^2
        sumM = sumM+absM
        sumM2 = sumM2+absM^2
        push!(maindf, (sumE/step,sumE2/step,sumM/step,sumM2/step))
    end
        
    return maindf
end

MonteCarlo_Ising (generic function with 2 methods)

# Q 1 a

In [ ]:
Ts = [1.0,1/3]
Ls = [16]
J = 1
N_chains = 10
n_steps = 1000000
warmup = Int(n_steps/10)

for L in Ls
    println("L = ",L)
    for T in Ts
        beta = 1/T
        println("T = ",T," and beta = ",beta)
        T = round(T,digits=3)
        for chain in 1:N_chains
            println("chain #",chain)
            @time begin
            full,avgs = MonteCarlo_Ising(beta,J,L,L,warmup,n_steps,build_lattice_df,build_exp_beta_deltaE_df,Metropolis_update_function,Calculate_Ising_EnergyPerSpin);
            end
            CSV.write("./data/q1a/L_$(L)/allvals_T_$(T)_chain$(chain).csv",  full, writeheader=true)
            CSV.write("./data/q1a/L_$(L)/avgvals_T_$(T)_chain$(chain).csv",  avgs, writeheader=true)
        end
    end
end

# Q 1 b

In [ ]:
# Ts = [0,0.0001,0.001,0.01,0.1,1,5,10,100,1000,10000,100000]
Ts = 0.0:0.2:4.0
Ls = [24]
J = 1
N_chains = 1
n_steps = 1000000
warmup = Int(n_steps/10)

for L in Ls
    println("L = ",L)
    for T in Ts
        beta = 1/T
        println("T = ",T," and beta = ",beta)
        T = round(T,digits=2)
        for chain in 1:N_chains
            println("chain #",chain)
            full,avgs = MonteCarlo_Ising(beta,J,L,L,warmup,n_steps,build_lattice_df,build_exp_beta_deltaE_df,Metropolis_update_function,Calculate_Ising_EnergyPerSpin);
            CSV.write("./data/q1b/L_$(L)/allvals_T_$(T)_chain$(chain).csv",  full, writeheader=true)
            CSV.write("./data/q1b/L_$(L)/avgvals_T_$(T)_chain$(chain).csv",  avgs, writeheader=true)
        end
    end
end

L = 24
T = 0.0 and beta = Inf
chain #1
Warmup steps over!
T = 0.2 and beta = 5.0
chain #1
Warmup steps over!
T = 0.4 and beta = 2.5
chain #1
Warmup steps over!
T = 0.6 and beta = 1.6666666666666667
chain #1
Warmup steps over!
T = 0.8 and beta = 1.25
chain #1
Warmup steps over!
T = 1.0 and beta = 1.0
chain #1
Warmup steps over!
T = 1.2 and beta = 0.8333333333333334
chain #1
Warmup steps over!
T = 1.4 and beta = 0.7142857142857143
chain #1
Warmup steps over!
T = 1.6 and beta = 0.625
chain #1
Warmup steps over!
T = 1.8 and beta = 0.5555555555555556
chain #1
Warmup steps over!
T = 2.0 and beta = 0.5
chain #1
Warmup steps over!
T = 2.2 and beta = 0.45454545454545453
chain #1
Warmup steps over!
T = 2.4 and beta = 0.4166666666666667
chain #1
Warmup steps over!
T = 2.6 and beta = 0.3846153846153846
chain #1
Warmup steps over!
T = 2.8 and beta = 0.35714285714285715
chain #1
Warmup steps over!
T = 3.0 and beta = 0.3333333333333333
chain #1
Warmup steps over!
T = 3.2 and beta = 0.3125
chain #1
Wa